# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [1]:
!pip install datasets  datasets==2.16.1 sentence-transformers torch

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [2]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
  'squad',
  split='train',
  streaming=True
).shuffle(seed=960)

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [3]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'id': '56bf8c8aa10cfb140055116f',
 'title': 'Beyoncé',
 'context': 'In July 2002, Beyoncé continued her acting career playing Foxxy Cleopatra alongside Mike Myers in the comedy film, Austin Powers in Goldmember, which spent its first weekend atop the US box office and grossed $73 million. Beyoncé released "Work It Out" as the lead single from its soundtrack album which entered the top ten in the UK, Norway, and Belgium. In 2003, Beyoncé starred opposite Cuba Gooding, Jr., in the musical comedy The Fighting Temptations as Lilly, a single mother whom Gooding\'s character falls in love with. The film received mixed reviews from critics but grossed $30 million in the U.S. Beyoncé released "Fighting Temptation" as the lead single from the film\'s soundtrack album, with Missy Elliott, MC Lyte, and Free which was also used to promote the film. Another of Beyoncé\'s contributions to the soundtrack, "Summertime", fared better on the US charts.',
 'question': "How did the critics view the movie

In [5]:
titles = [doc['title'] for doc in wiki_data]

In [6]:
from collections import Counter

title_counts = Counter(titles)

# Number of unique titles
num_unique_titles = len(title_counts)
print(f"Number of unique titles: {num_unique_titles}")

# Show top 10 most common titles
print(title_counts.most_common(40))

Number of unique titles: 442
[('New_York_City', 817), ('American_Idol', 802), ('Beyoncé', 758), ('Frédéric_Chopin', 697), ('Queen_Victoria', 680), ('Buddhism', 610), ('New_Haven,_Connecticut', 602), ('2008_Sichuan_earthquake', 521), ('2008_Summer_Olympics_torch_relay', 500), ('Muammar_Gaddafi', 489), ('Hellenistic_period', 469), ('Napoleon', 458), ('Middle_Ages', 452), ('Modern_history', 448), ('Portugal', 435), ('Gamal_Abdel_Nasser', 433), ('Dwight_D._Eisenhower', 430), ('Kanye_West', 428), ('Southampton', 426), ('The_Blitz', 414), ('Greece', 406), ('Religion_in_ancient_Rome', 405), ('Gramophone_record', 404), ('Dog', 392), ('Roman_Republic', 392), ('Pacific_War', 391), ('Financial_crisis_of_2007%E2%80%9308', 390), ('Mexico_City', 390), ('History_of_India', 389), ('Paris', 389), ('London', 379), ('Pub', 377), ('Tucson,_Arizona', 370), ('Protestantism', 369), ('Pharmaceutical_industry', 368), ('Plymouth', 367), ('Boston', 362), ('Age_of_Enlightenment', 361), ('Premier_League', 358), ('

In [7]:
# filter only documents with History as section_title - Replace None with your code
history = [doc for doc in wiki_data if 'History' in doc.get('title', '')]
print(f'Number of documents with "History" in title: {len(history)}')

Number of documents with "History" in title: 698


Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [8]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

historical_passages = []

# Total number of documents for tqdm
# total_doc_count = len(wiki_data)

# Iterate through the dataset
for doc in tqdm(wiki_data, total=total_doc_count, desc="Filtering historical passages"):
    # Check if 'history' appears in the context (case-insensitive)
    if 'history' in doc.get('context', '').lower():
        historical_passages.append({
            'article_title': doc.get('title', ''),
            # 'section_title': 'History',       # Assign History as section
            'passage_text': doc.get('context', '')
        })
    # Stop once we have 50,000 passages
    if len(historical_passages) > 50000:
        break

Filtering historical passages:   0%|          | 0/50000 [00:00<?, ?it/s]

In [9]:
print(len(historical_passages))

3743


In [10]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(historical_passages)
df.head()

,article_title,passage_text
0,Beyoncé,"A self-described ""modern-day feminist"", Beyonc..."
1,Beyoncé,"In August, the couple attended the 2011 MTV Vi..."
2,Beyoncé,"In August, the couple attended the 2011 MTV Vi..."
3,Beyoncé,"On December 13, 2013, Beyoncé unexpectedly rel..."
4,University_of_Notre_Dame,One of the main driving forces in the growth o...


In [11]:
!pip install pinecone

# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [16]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
print(api_key)

# configure client
pc = Pinecone(api_key="pcsk_6BUe3k_5E2SpCom69vAzzjDpMifRw7dzkkSeXM3Fw2jKeu9J46bgJ1Vrt3fbaXEDjmp4uY")

PINECONE_API_KEY


Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [17]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [14]:
index_name = "abstractive-question-answering" #give your index a meaningful name

In [18]:
import time

dimension_size = 768
metric_type = "cosine"

# check if the extractive-question-answering index exists
if index_name not in pc.list_indexes().names():
    # create the index if it does not exist
    pc.create_index(
        name=index_name,
        dimension=dimension_size,
        metric=metric_type,
        # Specify Serverless configuration
        spec=spec
    )
# connect to extractive-question-answering index we created
index = pc.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [19]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
model_name = "flax-sentence-embeddings/all_datasets_v3_mpnet-base"

retriever = SentenceTransformer(model_name, device=device) #load the retriever model from HuggingFace. Use the flax-sentence-embeddings/all_datasets_v3_mpnet-base model
retriever

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [ ]:
# we will use batches of 64
batch_size = 64

batch_size = 64

all_vectors_to_upsert = [] # Moved initialization outside the loop to accumulate correctly

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    end = min(i + batch_size, len(df))
    # extract batch
    batch = df.iloc[i:end]
    # generate embeddings for batch
    contexts_to_embed = batch['context'].tolist()
    emb = retriever.encode(contexts_to_embed).tolist()
    # get metadata
    meta = batch.to_dict(orient='records')
    meta_cleaned = []
    for record in meta:
        meta_cleaned.append({
            "context": record['context'],
            "title": record['title']
            # Add any other relevant fields like 'question', 'answer_text' if needed later
        })
    meta = meta_cleaned # Use the cleaned metadata
    # create unique IDs
    ids = [f"doc_{j}" for j in range(i, end)]
    # add all to upsert list
    to_upsert = []
    for doc_id, vector, metadata_dict in zip(ids, emb, meta):
        to_upsert.append({
            "id": doc_id,
            "values": vector,
            "metadata": metadata_dict
        })
    # Optional: collect all for debugging (keep this line indented)
    all_vectors_to_upsert.extend(to_upsert) # This accumulates data from all batches

    # upsert/insert these records to pinecone
    # The `index` object is what you got from `index = pc.Index(name)`
    _ = index.upsert(vectors=to_upsert) # <--- CORRECT INDENTATION: This call should be inside the loop for each batch


# check that we have all vectors in index
# These lines should be outside the loop to be run once after all upserts are complete
index_stats = index.describe_index_stats()
print(index_stats)

# verify the count from your DataFrame
print(f"Expected number of vectors (from df): {len(df)}")
index.describe_index_stats()

  0%|          | 0/782 [00:00<?, ?it/s]

{'dimension': 768,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 50001}},
 'total_vector_count': 50001}

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [ ]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = None
    # search pinecone index for context passage with the answer
    xc = None
    return xc

In [ ]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = None
    # contcatinate the query and context passages
    query = None
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [25]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

NameError: name 'query_pinecone' is not defined

In [ ]:
from pprint import pprint

In [ ]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
pprint(query)

('question: when was the first electric power system built? context: <P> '
 "Electric power system History In 1881, two electricians built the world's "
 'first power system at Godalming in England. It was powered by two '
 'waterwheels and produced an alternating current that in turn supplied seven '
 'Siemens arc lamps at 250 volts and 34 incandescent lamps at 40 volts. '
 'However, supply to the lamps was intermittent and in 1882 Thomas Edison and '
 'his company, The Edison Electric Light Company, developed the first '
 'steam-powered electric power station on Pearl Street in New York City. The '
 'Pearl Street Station initially powered around 3,000 lamps for 59 customers. '
 'The power station generated direct current and')


The output looks great. Now let's write a function to generate answers.

In [ ]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [ ]:
generate_answer(query)

('The first electric power system was built in 1881 at Godalming in England. '
 'It was powered by two waterwheels and produced alternating current that in '
 'turn supplied seven Siemens arc lamps')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [ ]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The first wireless message was sent in 1866 by Mahlon Loomis, who had a kite '
 'on a mountaintop 14 miles apart. The kite was connected to a cable')


To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [ ]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

by electrostatic induction or electromagnetic induction, which had too short a range to be practical. In 1866 Mahlon Loomis claimed to have transmitted an electrical signal through the atmosphere between two 600 foot wires held aloft by kites on mountaintops 14 miles apart. Thomas Edison had come close to discovering radio in 1875; he had generated and detected radio waves which he called "etheric currents" experimenting with high-voltage spark circuits, but due to lack of time did not pursue the matter. David Edward Hughes in 1879 had also stumbled on radio wave transmission which he received with his carbon microphone
---
the east coast of India, then on to Penang, Malacca, Singapore, Batavia (current Jakarta), to finally reach Darwin, Australia. It was the first direct link between Australia and Great Britain. The company that laid the first part of the cable took the name of Falmouth, Gibraltar and Malta Telegraph Company and had been founded in 1869. This company later operated as

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [ ]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('COVID-19 is a zoonotic disease, which means that it is a virus that is '
 'transmitted from one animal to another. It is not a virus that can be '
 'transmitted from person')


In [ ]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

to establish with certainty which diseases jumped from other animals to humans, but there is increasing evidence from DNA and RNA sequencing, that measles, smallpox, influenza, HIV, and diphtheria came to humans this way. Various forms of the common cold and tuberculosis also are adaptations of strains originating in other species.
Zoonoses are of interest because they are often previously unrecognized diseases or have increased virulence in populations lacking immunity. The West Nile virus appeared in the United States in 1999 in the New York City area, and moved through the country in the summer of 2002, causing much distress. Bubonic
---
plague is a zoonotic disease, as are salmonellosis, Rocky Mountain spotted fever, and Lyme disease.
A major factor contributing to the appearance of new zoonotic pathogens in human populations is increased contact between humans and wildlife. This can be caused either by encroachment of human activity into wilderness areas or by movement of wild ani

Let’s finish with a final few questions.

In [ ]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

('The War of Currents was a series of events in the early 1900s between Edison '
 'and Westinghouse. The two companies were competing for the market share of '
 'electric power in the United States')


In [ ]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

('The first person to walk on the moon was Neil Armstrong in 1969. He walked '
 'on the moon in 1969. He was the first person to walk on the moon.')


In [ ]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

('The Space Shuttle was the most expensive project in the history of NASA. It '
 'cost about $10 billion to build.')


As we can see, the model can generate some decent answers.

#### Add a few more questions